In [1]:

# Preparing Data
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import*
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
x_data = x_data.drop('id', axis=1)
y_data = df['target']
x_data['encoded_column'] = (x_data['sensor_2'] < 0.02).astype(int)

def transform_continuous_columns(df, operation='sum'):
    # Identify continuous columns (numeric types)
    continuous_cols = df.select_dtypes(include=['number']).columns
    
    # Initialize a dictionary to store new columns
    new_columns = {}
    
    # Iterate over pairs of continuous columns
    for i in range(len(continuous_cols)):
        for j in range(i + 1, len(continuous_cols)):
            col1 = continuous_cols[i]
            col2 = continuous_cols[j]
            new_col_name = f"{col1}{operation}{col2}"
            
            if operation == '+':
                new_columns[new_col_name] = df[col1] + df[col2]
            elif operation == '*':
                new_columns[new_col_name] = df[col1] * df[col2]
            elif operation == '-':
                new_columns[new_col_name] = df[col1] - df[col2]
            elif operation == '/':
                new_columns[new_col_name] = df[col1] / df[col2]
            else:
                raise ValueError("Unsupported operation.")
    
    # Create a new DataFrame from the collected columns
    result_df = pd.DataFrame(new_columns, index=df.index)
    
    return result_df
# Applying the function
df_sum = transform_continuous_columns(x_data, operation='+')
df_multi = transform_continuous_columns(x_data, operation='*')
df_subtract = transform_continuous_columns(x_data, operation='-')
df_division = transform_continuous_columns(x_data, operation='/')

df_transformed_multivariate = pd.concat([df_sum, df_multi, df_subtract, df_division], axis=1)
